# Lab 7, 15.04. Bayes spell checker

Goals: 
* get a little bit more comfortable with writing in Python 
* understand model training and evaluation
    * will be very useful for projects (for instance simple model for fitting missing data or making predictions about user)

Content inspired by Peter Norvig post about spell checker

<img src="https://imgs.xkcd.com/comics/frequentists_vs_bayesians.png">

# Bayes spell checker - theory

Let:
* w - misspelled word
* c - candidate word

We want to find $ argmax_c P(c | w) $

This is the same from Bayes rule as $ argmax_c p(w | c) p(c) $. 

We will simplify and say $p(w | c)$ (our **error model**) is just "pick edit distance 1 if found, if not resort to edit distance 2"

## More about text files

* We already know csv format. There are also json and yaml formats that are common
* Fetching all files using glob

## Excercise 1. 

* Write function that will read all words from data/7/big.txt (using regex, remember to lower text)
* Plot plot of counts, horizontal axis is ordered index and vertical axis is log of word count. This should look like this https://en.wikipedia.org/wiki/Zipf%27s_law

In [29]:
WORDS # = ?? - your code

assert len(WORDS) == 131810 

NameError: name 'WORDS' is not defined

In [ ]:
from collections import Counter
COUNTS = Counter(WORDS)

# Exercise 2 [1 - 2 points]

* Write function edits0(word) that returns only {word} and edits1(word) that returns **set** of all words being 1 edit away from our word
* 2 points if the function is implemented in 6 lines (see Useful code)

In [ ]:
assert edits0('cat') == {'cat'}
ed1_Cat = set(['caqt', 'ucat', 'cdt', 'ctat', 'ciat', 'vcat', 'cvat', 'ycat', 'caht', 'cut', 'jat', 
               'caty', 'clt', 'hat', 'cyat', 'capt', 'icat', 'zcat', 'fat', 'dat', 'cet', 
               'caot', 'catz', 'hcat', 'bat', 'crt', 'cayt', 'cakt', 'clat', 'cmt', 'cvt', 
               'ceat', 'cwat', 'cjat', 'cnat', 'acat', 'cft', 'cabt', 'cnt', 'cajt', 'aat', 
               'cwt', 'cast', 'czat', 'csat', 'cqat', 'cit', 'cart', 'jcat', 'cfat', 'cazt', 
               'pcat', 'catd', 'caat', 'cgt', 'ctt', 'cati', 'cait', 'cot', 'cawt', 'xcat', 
               'cta', 'act', 'ncat', 'cxt', 'ckat', 'calt', 'ca', 'dcat', 'cadt', 'zat', 
               'cato', 'ct', 'crat', 'cata', 'catb', 'catc', 'tcat', 'cate', 'catf', 'catg', 
               'cath', 'yat', 'catj', 'catk', 'xat', 'catm', 'catn', 'catl', 'catp', 'ocat', 
               'catr', 'cats', 'cht', 'catu', 'catv', 'catw', 'catx', 'iat', 'bcat', 'wat', 
               'catq', 'vat', 'cqt', 'cact', 'cyt', 'rcat', 'gat', 'cant', 'cgat', 'mcat', 
               'eat', 'kcat', 'caz', 'cay', 'cax', 'cas', 'car', 'caq', 'cap', 'caw', 'cav', 
               'cau', 'cat', 'cak', 'caj', 'cai', 'cah', 'cao', 'can', 'cam', 'cal', 'cac', 
               'cab', 'caa', 'cag', 'caf', 'cae', 'cad', 'tat', 'chat', 'fcat', 'caft', 'lcat', 
               'uat', 'czt', 'rat', 'at', 'cbt', 'catt', 'scat', 'sat', 'qat', 'qcat', 'pat', 
               'wcat', 'cuat', 'oat', 'nat', 'cst', 'cavt', 'cjt', 'mat', 'cxat', 'caet', 'cmat', 
               'ccat', 'cagt', 'cpat', 'kat', 'lat', 'gcat', 'caxt', 'cdat', 'coat', 'cct', 'camt', 
               'ckt', 'caut', 'cpt', 'cbat', 'ecat']) 
edits1('cat') == ed1_Cat

In [ ]:
# Useful code

def splits(word):
    "Return a list of all possible (first, rest) pairs that comprise word."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

def edits1(word):
    pairs      = splits(word)
    # 1. Calculate all transpose cat -> cta
    # 2. Calculate all deletes cat -> at
    # 3. Calculate all replaces cat -> cst
    # 4. Calculate all inserts cat -> caxt
    # Return sum

alphabet = 'abcdefghijklmnopqrstuvwxyz'

# Exercise 3, Spell checker!

Write spellchecker, as:
    * Check if word is in COUNTS and if yes return {word}
    * Check if edits1(word) is in COUNTS and if yes return all predictions
    * Check if edits2(word) is in COUNTS and if yes return all predictions

In [13]:
assert correct('economtric') == 'economic'
assert correct('zcat') == 'cat'
assert correct('cmoing') == 'coming'
assert correct("thay") == "that", "That is more probable than they"

NameError: global name 'candidates' is not defined

In [ ]:
## Useful code

def edits2(word):
    "Return all strings that are two edits away from this word."
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

def known(words):
    "Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in COUNTS}

def correct(word):
    "Find the best spelling correction for this word. That is argmax_c p(c), where c are candidates"
    # Prefer edit distance 0, then 1, then 2; otherwise default to word itself.
    # [Fill in] #
    # Return most probable candidate (P(c))
    return # ? Use COUNTS to select most probable word

# Actual spell-checker

In [ ]:
import re

def correct_text(text):
    "Correct all the words within a text, returning the corrected text."
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    "Spell-correct word in match, and preserve proper upper/lower/title case."
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    "Return the case-function appropriate for text: upper, lower, title, or just str."
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

assert correct_text('Speling Errurs IN somethink. Whutever; unusuel misteakes?') == \
    'Seeing Errors IN something. Whatever; unusual mistaken?'

# Evaluation

What does it mean to evaluate model?

In [ ]:
tests1 = json.load(open("data/7/test1.json"))
tests2 = json.load(open("data/7/test2.json"))

def spelltest(tests, verbose=False):
    import time
    n, bad, unknown, start = 0, 0, 0, time.clock()
    for target,wrongs in tests.items():
        for wrong in wrongs.split():
            n += 1
            w = correct(wrong)
            if w!=target:
                bad += 1
                unknown += (target not in COUNTS)
                if verbose:
                    print '%r => %r (%d); expected %r (%d)' % (
                        wrong, w, NWORDS[w], target, NWORDS[target])
    return dict(bad=bad, n=n, bias=bias, pct=int(100. - 100.*bad/n), 
                unknown=unknown, secs=int(time.clock()-start) )

print spelltest(tests1)
print spelltest(tests2) ## only do this after everything is debugged

# Miniproject 2, Try to improve spellchecker

For ideas see http://norvig.com/spell-correct.html , Future work. Try to implement one of the ideas